In [51]:
import torch
import pandas as pd
from torch.profiler import profile, record_function, ProfilerActivity

torch.random.manual_seed(0)

print(torch.__version__)


2.0.0.dev20230213+cu117


In [52]:
SIZES = [128, 256, 512, 1024, 2048, 4096, 8192, 16384]
PERCENTAGES_NONZERO = [0.01, 0.05, 0.1, 0.2, 0.5]
DEVICE = "cuda"

densify = lambda x: x.to_dense() if x.is_sparse else x
cooify = lambda x : x.to_sparse_coo()
cscify = lambda x : x.to_sparse_csc()
csrify = lambda x : x.to_sparse_csr()

MATRIX_TYPES = [
    ("dense", densify),
    ("coo", cooify),
    ("csc", cscify),
    ("csr", csrify),
]


In [53]:
results = []
for size in SIZES:
    b = torch.rand((size, 1), device=DEVICE)
    for percentage_nonzero in PERCENTAGES_NONZERO:
        A_dense = torch.rand((size, size), device=DEVICE)
        mask = torch.rand((size, size), device=DEVICE) > percentage_nonzero
        A_dense = A_dense.masked_fill_(mask, 0)
        archieved_percentage = 1 - (mask.sum() / (size * size))
        print(f"Running {size}x{size} with {percentage_nonzero}% non-zero (achieved {archieved_percentage}%)")

        for name, matrix_fun in MATRIX_TYPES:
            matrix = matrix_fun(A_dense)
            with profile(activities=[ProfilerActivity.CUDA], with_stack=True) as prof:
                _ = torch.matmul(matrix, b)
            prof.export_chrome_trace(f"torch_sparse_trace_{name}.json")
            prof.export_stacks(f"profiler_stacks_{name}.txt", "self_cuda_time_total")

            result = {
                'size': size,
                'percentage_nonzero': percentage_nonzero,
                'name': name,
                'cuda_str': prof.key_averages().total_average().self_cuda_time_total_str,
                'cuda_micros': prof.key_averages().total_average().self_cuda_time_total,
                'cpu_str': prof.key_averages().total_average().self_cpu_time_total_str,
                'cpu_micros': prof.key_averages().total_average().self_cuda_time_total
            }
            results.append(result)

df = pd.DataFrame(results)

Running 128x128 with 0.01% non-zero (achieved 0.00970458984375%)
Running 128x128 with 0.05% non-zero (achieved 0.05078125%)
Running 128x128 with 0.1% non-zero (achieved 0.10107421875%)
Running 128x128 with 0.2% non-zero (achieved 0.20257568359375%)
Running 128x128 with 0.5% non-zero (achieved 0.501220703125%)
Running 256x256 with 0.01% non-zero (achieved 0.01019287109375%)
Running 256x256 with 0.05% non-zero (achieved 0.0507965087890625%)
Running 256x256 with 0.1% non-zero (achieved 0.100311279296875%)
Running 256x256 with 0.2% non-zero (achieved 0.202911376953125%)
Running 256x256 with 0.5% non-zero (achieved 0.4988250732421875%)
Running 512x512 with 0.01% non-zero (achieved 0.009960174560546875%)
Running 512x512 with 0.05% non-zero (achieved 0.0500335693359375%)


STAGE:2023-02-17 01:45:33 120517:120517 ActivityProfilerController.cpp:311] Completed Stage: Warm Up
STAGE:2023-02-17 01:45:33 120517:120517 ActivityProfilerController.cpp:317] Completed Stage: Collection
STAGE:2023-02-17 01:45:33 120517:120517 ActivityProfilerController.cpp:321] Completed Stage: Post Processing
STAGE:2023-02-17 01:45:33 120517:120517 ActivityProfilerController.cpp:311] Completed Stage: Warm Up
STAGE:2023-02-17 01:45:33 120517:120517 ActivityProfilerController.cpp:317] Completed Stage: Collection
STAGE:2023-02-17 01:45:33 120517:120517 ActivityProfilerController.cpp:321] Completed Stage: Post Processing
STAGE:2023-02-17 01:45:33 120517:120517 ActivityProfilerController.cpp:311] Completed Stage: Warm Up
STAGE:2023-02-17 01:45:33 120517:120517 ActivityProfilerController.cpp:317] Completed Stage: Collection
STAGE:2023-02-17 01:45:33 120517:120517 ActivityProfilerController.cpp:321] Completed Stage: Post Processing
STAGE:2023-02-17 01:45:33 120517:120517 ActivityProfilerCo

Running 512x512 with 0.1% non-zero (achieved 0.09929275512695312%)
Running 512x512 with 0.2% non-zero (achieved 0.1996917724609375%)
Running 512x512 with 0.5% non-zero (achieved 0.5016212463378906%)
Running 1024x1024 with 0.01% non-zero (achieved 0.010016441345214844%)
Running 1024x1024 with 0.05% non-zero (achieved 0.05009269714355469%)
Running 1024x1024 with 0.1% non-zero (achieved 0.1002664566040039%)
Running 1024x1024 with 0.2% non-zero (achieved 0.20029735565185547%)
Running 1024x1024 with 0.5% non-zero (achieved 0.49987030029296875%)
Running 2048x2048 with 0.01% non-zero (achieved 0.010030269622802734%)


STAGE:2023-02-17 01:45:34 120517:120517 ActivityProfilerController.cpp:311] Completed Stage: Warm Up
STAGE:2023-02-17 01:45:34 120517:120517 ActivityProfilerController.cpp:317] Completed Stage: Collection
STAGE:2023-02-17 01:45:34 120517:120517 ActivityProfilerController.cpp:321] Completed Stage: Post Processing
STAGE:2023-02-17 01:45:34 120517:120517 ActivityProfilerController.cpp:311] Completed Stage: Warm Up
STAGE:2023-02-17 01:45:34 120517:120517 ActivityProfilerController.cpp:317] Completed Stage: Collection
STAGE:2023-02-17 01:45:34 120517:120517 ActivityProfilerController.cpp:321] Completed Stage: Post Processing
STAGE:2023-02-17 01:45:34 120517:120517 ActivityProfilerController.cpp:311] Completed Stage: Warm Up
STAGE:2023-02-17 01:45:34 120517:120517 ActivityProfilerController.cpp:317] Completed Stage: Collection
STAGE:2023-02-17 01:45:34 120517:120517 ActivityProfilerController.cpp:321] Completed Stage: Post Processing
STAGE:2023-02-17 01:45:34 120517:120517 ActivityProfilerCo

Running 2048x2048 with 0.05% non-zero (achieved 0.05009579658508301%)
Running 2048x2048 with 0.1% non-zero (achieved 0.10026979446411133%)
Running 2048x2048 with 0.2% non-zero (achieved 0.19993972778320312%)
Running 2048x2048 with 0.5% non-zero (achieved 0.5002305507659912%)
Running 4096x4096 with 0.01% non-zero (achieved 0.009996473789215088%)
Running 4096x4096 with 0.05% non-zero (achieved 0.049968838691711426%)
Running 4096x4096 with 0.1% non-zero (achieved 0.10005909204483032%)
Running 4096x4096 with 0.2% non-zero (achieved 0.20003104209899902%)
Running 4096x4096 with 0.5% non-zero (achieved 0.4999313950538635%)


...

STAGE:2023-02-17 01:45:35 120517:120517 ActivityProfilerController.cpp:311] Completed Stage: Warm Up
STAGE:2023-02-17 01:45:35 120517:120517 ActivityProfilerController.cpp:317] Completed Stage: Collection
STAGE:2023-02-17 01:45:35 120517:120517 ActivityProfilerController.cpp:321] Completed Stage: Post Processing
STAGE:2023-02-17 01:45:35 120517:120517 ActivityProfilerController.cpp:311] Completed Stage: Warm Up
STAGE:2023-02-17 01:45:35 120517:120517 ActivityProfilerController.cpp:317] Completed Stage: Collection
STAGE:2023-02-17 01:45:35 120517:120517 ActivityProfilerController.cpp:321] Completed Stage: Post Processing
STAGE:2023-02-17 01:45:35 120517:120517 ActivityProfilerController.cpp:311] Completed Stage: Warm Up
STAGE:2023-02-17 01:45:35 120517:120517 ActivityProfilerController.cpp:317] Completed Stage: Collection
STAGE:2023-02-17 01:45:35 120517:120517 ActivityProfilerController.cpp:321] Completed Stage: Post Processing
STAGE:2023-02-17 01:45:35 120517:120517 ActivityProfilerCo

Running 16384x16384 with 0.2% non-zero (achieved 0.20001119375228882%)


STAGE:2023-02-17 01:45:35 120517:120517 ActivityProfilerController.cpp:317] Completed Stage: Collection
STAGE:2023-02-17 01:45:35 120517:120517 ActivityProfilerController.cpp:321] Completed Stage: Post Processing
STAGE:2023-02-17 01:45:35 120517:120517 ActivityProfilerController.cpp:311] Completed Stage: Warm Up
STAGE:2023-02-17 01:45:35 120517:120517 ActivityProfilerController.cpp:317] Completed Stage: Collection
STAGE:2023-02-17 01:45:35 120517:120517 ActivityProfilerController.cpp:321] Completed Stage: Post Processing
STAGE:2023-02-17 01:45:35 120517:120517 ActivityProfilerController.cpp:311] Completed Stage: Warm Up
STAGE:2023-02-17 01:45:35 120517:120517 ActivityProfilerController.cpp:317] Completed Stage: Collection
STAGE:2023-02-17 01:45:35 120517:120517 ActivityProfilerController.cpp:321] Completed Stage: Post Processing
STAGE:2023-02-17 01:45:35 120517:120517 ActivityProfilerController.cpp:311] Completed Stage: Warm Up
STAGE:2023-02-17 01:45:35 120517:120517 ActivityProfilerCo

Running 16384x16384 with 0.5% non-zero (achieved 0.49997055530548096%)


STAGE:2023-02-17 01:45:35 120517:120517 ActivityProfilerController.cpp:311] Completed Stage: Warm Up
STAGE:2023-02-17 01:45:36 120517:120517 ActivityProfilerController.cpp:317] Completed Stage: Collection
STAGE:2023-02-17 01:45:36 120517:120517 ActivityProfilerController.cpp:321] Completed Stage: Post Processing
STAGE:2023-02-17 01:45:36 120517:120517 ActivityProfilerController.cpp:311] Completed Stage: Warm Up
STAGE:2023-02-17 01:45:36 120517:120517 ActivityProfilerController.cpp:317] Completed Stage: Collection
STAGE:2023-02-17 01:45:36 120517:120517 ActivityProfilerController.cpp:321] Completed Stage: Post Processing


In [54]:
df

,size,percentage_nonzero,name,cuda_str,cuda_micros,cpu_str,cpu_micros
0,128,0.01,dense,3.000us,3,384.000us,3
1,128,0.01,coo,26.000us,26,398.000us,26
2,128,0.01,csc,109.000us,109,498.000us,109
3,128,0.01,csr,12.000us,12,344.000us,12
4,128,0.05,dense,2.000us,2,311.000us,2
...,...,...,...,...,...,...,...
155,16384,0.20,csr,1.599ms,1599,14.365ms,1599
156,16384,0.50,dense,2.636ms,2636,2.873ms,2636
157,16384,0.50,coo,12.201ms,12201,48.958ms,12201
158,16384,0.50,csc,271.676ms,271676,305.575ms,271676
